In [1]:
import sqlite3
import pandas as pd

### Подключение к БД

In [2]:
conn = sqlite3.connect("data.db")

Выбираем все договоры самой старшей версии с меткой emergency - "Закупка осуществляется вследствие аварии,
 иных чрезвычайных ситуаций природного или техногенного характера, непреодолимой силы,
 принеобходимости срочного медицинского вмешательства, а также для предотвращения угрозы
 возникновения указанных ситуаций"

In [3]:
covid_contracts = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    """,
    conn
)

covid_contracts

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,purchase_notice_info_number,lot_guid,subject_contract,purchase_type_code,contract_price,contract_rub_price,contract_currency_code,contract_start_execution_date,contract_end_execution_date,region_name
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,1,0,0,None,None,2020-09-08 07:01:28,7825011824,...,32009471555,34579301-6678-4603-bc7f-372389912bb6,Выполнение аварийно-восстановительных работ по...,40000,5.904899e+05,NaN,RUB,2020-09-08,2020-11-29,Sankt-Peterburg
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,0,0,None,None,2021-08-17 14:12:13,6901067107,...,32110475167,15ed771e-7b12-b10b-cb82-0394460efb4b,СМР и ПНР: 1 очередь строительства распределит...,40000,2.108040e+06,NaN,RUB,2021-08-12,2021-09-15,Moskva
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,0,0,None,None,2020-06-05 07:00:39,2312210286,...,32009195991,15fd6ba3-3df0-41da-b2ce-e2a8fef5800c,Ремонт уличного водопровода по ул.Западная от ...,30000,2.924070e+05,NaN,RUB,2020-03-15,None,Krasnodarskii_krai
3,0118391a-07a3-4ac6-b148-d35a087b676c,55053031065210000300001,0,0,0,0,None,None,2021-05-12 16:25:06,5053031065,...,32110238087,1ee63bc8-6be4-4e83-affd-49fff95065b1,Выполнение работ по устройству закрытого перех...,30000,9.954500e+05,NaN,RUB,2021-05-10,2021-06-29,Moskovskaya_obl
4,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,0,0,None,None,2020-06-23 06:16:18,7703695246,...,32009227466,4b15db7b-bed4-46ae-b5b4-f66aecd3f2a7,1400-2020-00102 Поставка аппаратов для искусст...,40000,3.318084e+07,33180840.0,RUB,2020-04-22,2020-12-30,Moskva
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,0,0,None,None,2020-06-18 13:00:13,7703695246,...,32009227271,5a7a5527-faf9-4e0b-9e42-de59e14d1fa9,1400-2020-00063 Поставка аппаратов для искусст...,40000,5.530140e+06,5530140.0,RUB,2020-04-16,None,Moskva
1643,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,0,0,None,None,2020-11-18 07:20:58,7728816598,...,32009691019,04da8eeb-458e-4f1c-bd56-ddb591c73fc7,1004-2020-01351 Поставка пироэлектрического да...,40000,1.848000e+05,184800.0,RUB,2020-11-16,2020-12-30,Moskva
1644,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,0,0,None,None,2020-07-03 13:49:05,7703695246,...,32009227418,91773610-ce48-4282-aded-c069f605f8c0,1400-2020-00059 Поставка аппаратов для искусст...,40000,3.375900e+08,337590000.0,RUB,2020-04-02,2020-12-30,Moskva
1645,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,32009775118,dcf4b21c-2825-49d7-9611-f9aeb85b18b6,Ремонт водопровода по ул.Проточной от жилого д...,30000,2.949280e+05,NaN,RUB,2020-07-29,None,Krasnodarskii_krai


Для дальнейшего анализа необходимо проверить наличие поля с указанием лота

In [4]:
covid_contracts_with_not_null_lot = covid_contracts[covid_contracts["lot_guid"].notnull()]

print(f'Количество "ковидных" договоров: {covid_contracts.shape[0]}')
print(f'Количество "ковидных" договоров с указанием лота: {covid_contracts_with_not_null_lot.shape[0]}')

Количество "ковидных" договоров: 1647
Количество "ковидных" договоров с указанием лота: 1647


Договоры без указания lot_guid, это те где извещение не размещалось в соответствии
с частью 5 статьи 4 Федерального Закона № 223-ФЗ, например https://zakupki.gov.ru/epz/contractfz223/card/contract-info.html?id=9268727

Для выборки будем использовать только те договоры, в которых присутствуют лоты

Проверим, существуют ли лоты, на которые было заключено несколько договоров.

In [5]:
lots_with_many_contracts = pd.read_sql_query(
    """
    SELECT * FROM (
        SELECT lot_guid, COUNT(*) as c
            FROM contract c
            INNER JOIN (
                SELECT contract_reg_number, MAX(version) ver
                FROM contract
                WHERE emergency = 1
                and lot_guid is not null
                GROUP BY contract_reg_number
            ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
        GROUP BY lot_guid
    ) WHERE c > 1
    """,
    conn
)

lots_with_many_contracts

,lot_guid,c
0,0a0bad0e-049a-43e9-adfc-6896a21b42ed,3
1,11db9fb9-da1b-49fc-a31e-97462414707f,34
2,1581b86c-4c98-40d3-b578-76c76f4a2670,2
3,1c475aea-1c94-4fb8-913f-54d61fc719ed,23
4,3032623b-bcf8-4b91-b93d-03010e4a41c3,2
5,44c059df-9364-4dc7-83cd-3cde676c82fc,25
6,7a90a090-6157-4c32-b276-b68f6643d077,2
7,8885e84c-8f87-4723-905b-4e927a1ce195,36
8,89702ad6-8d7c-4515-aaa3-f19d400aa2b4,3
9,bd47d4a1-a936-47d9-aab1-5617fd5f3243,2


Такие договоры существуют.  Необходимо будет добавить информацию об этом в выборку.

Выберем из базы договоры с лотами

In [6]:
covid_contracts_with_lot = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    """,
    conn
)

covid_contracts_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,contract_start_execution_date,contract_end_execution_date,region_name,notice_lot_initial_sum,notice_lot_starting_contract_price_rub,notice_lot_currency_code,notice_lot_commodity_price,notice_lot_commodity_price_rub,notice_lot_max_contract_price,notice_lot_max_contract_price_rub
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,1,0,0,None,None,2020-09-08 07:01:28,7825011824,...,2020-09-08,2020-11-29,Sankt-Peterburg,5.904899e+05,NaN,RUB,None,None,NaN,None
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,0,0,None,None,2021-08-17 14:12:13,6901067107,...,2021-08-12,2021-09-15,Moskva,2.173348e+06,NaN,RUB,None,None,NaN,None
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,0,0,None,None,2020-06-05 07:00:39,2312210286,...,2020-03-15,None,Krasnodarskii_krai,2.924070e+05,NaN,RUB,None,None,NaN,None
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,0,0,None,None,2020-06-23 06:16:18,7703695246,...,2020-04-22,2020-12-30,Moskva,3.318084e+07,NaN,RUB,None,None,NaN,None
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,0,0,None,None,2021-06-09 15:04:20,7813417393,...,2021-04-20,2021-12-30,Sankt-Peterburg,2.500000e+07,NaN,RUB,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,0,0,None,None,2020-06-18 13:00:13,7703695246,...,2020-04-16,None,Moskva,5.530140e+06,NaN,RUB,None,None,NaN,None
1537,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,0,0,None,None,2020-11-18 07:20:58,7728816598,...,2020-11-16,2020-12-30,Moskva,1.848000e+05,NaN,RUB,None,None,NaN,None
1538,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,0,0,None,None,2020-07-03 13:49:05,7703695246,...,2020-04-02,2020-12-30,Moskva,3.341800e+08,NaN,RUB,None,None,NaN,None
1539,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,2020-07-29,None,Krasnodarskii_krai,2.949280e+05,NaN,RUB,None,None,NaN,None


Проверим, существуют ли договоры, у которых заказчик отличается от заказчика в извещении

In [7]:
contracts_with_different_customer = pd.read_sql_query(
    """
    SELECT c.guid, c.customer_inn, c.placer_inn, c.detached_org_inn, pn.guid, pn.customer_inn, pn.placer_inn, pn.detached_org_inn
    FROM contract c
    INNER JOIN purchase_notice_lot pnl on c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_to_lot pntl on pnl.guid = pntl.purchase_notice_lot_guid
    LEFT JOIN purchase_notice pn on pntl.purchase_notice_guid = pn.guid
    WHERE
    c.lot_guid is not null
    AND c.emergency = 1
    AND (c.customer_inn != pn.customer_inn or c.placer_inn != pn.placer_inn)
    ;
    """,
    conn
)

contracts_with_different_customer

,guid,customer_inn,placer_inn,detached_org_inn,guid,customer_inn,placer_inn,detached_org_inn
0,02a74aef-0dc0-48fb-b0d3-70731900673d,7736182930,7736182930,7736182930,84925ecd-c713-420e-b1a4-683af576b9e9,7736182930,7707308480,None
1,06c265e1-678b-4668-a263-54c8a9bcf63b,6345012488,6345012488,6345012488,1d063c20-7aa0-46ad-9289-a1b9bfb28b87,1510012774,1510012774,None
2,06c265e1-678b-4668-a263-54c8a9bcf63b,6345012488,6345012488,6345012488,219cfb41-11cb-aae8-b709-0d5a6bd131db,1510012774,1510012774,None
3,0c62cca5-11ff-4c74-98b3-99bfba43f439,7736182930,7736182930,7736182930,86da896b-2ac2-40d9-b8fd-5afc5c6834ef,7736182930,7707308480,None
4,1075c209-6479-4383-b2f9-5094f6f1d5ac,7736182930,7736182930,7736182930,7ecca033-8e5c-4fc4-b14f-a73593cea657,7736182930,7707308480,None
...,...,...,...,...,...,...,...,...
59,eef8ca67-51e9-4504-a782-3553ee146548,7729050901,7729050901,7729050901,d0397010-c8c5-4a93-be51-dfcb47699583,7729050901,7707308480,None
60,ef7f48fb-1db6-49de-88bd-9797dcaa6a63,7736182930,7736182930,7736182930,c4e42361-67c8-443b-a217-422ccdd9d91e,7736182930,7707308480,None
61,f151c684-068f-483d-b4d8-ea83f6133261,7736182930,7736182930,7736182930,78c3f1f2-4e8d-41e6-9379-1b40a96dc59a,7736182930,7707308480,None
62,fe33caf4-1d35-4e0a-80ba-9706164e3018,2308131994,2308131994,2308131994,ff0817d0-30ce-4169-921e-75834f01b2d2,2308131994,7707308480,None


Такие договоры существуют, поэтому необходимо будет добавить данные об изначальном заказчике в выборку

Дополним информацией о том, был ли договор завершен или расторгнут

In [8]:
covid_contracts_performance_with_lot = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub,
    cp.is_complete as contract_is_complete
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN (
        SELECT c2.contract_reg_number, MIN(is_complete) as is_complete
        FROM contract c2
        LEFT JOIN contract_performance cp2 ON c2.contract_reg_number = cp2.contract_registration_number
        WHERE c2.emergency = 1
        AND c2.lot_guid is not null
        GROUP BY c2.contract_reg_number
    ) cp ON cp.contract_reg_number = c.contract_reg_number
    """,
    conn
)

covid_contracts_performance_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,contract_end_execution_date,region_name,notice_lot_initial_sum,notice_lot_starting_contract_price_rub,notice_lot_currency_code,notice_lot_commodity_price,notice_lot_commodity_price_rub,notice_lot_max_contract_price,notice_lot_max_contract_price_rub,contract_is_complete
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,1,0,0,None,None,2020-09-08 07:01:28,7825011824,...,2020-11-29,Sankt-Peterburg,5.904899e+05,NaN,RUB,None,None,NaN,None,1.0
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,0,0,None,None,2021-08-17 14:12:13,6901067107,...,2021-09-15,Moskva,2.173348e+06,NaN,RUB,None,None,NaN,None,NaN
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,0,0,None,None,2020-06-05 07:00:39,2312210286,...,None,Krasnodarskii_krai,2.924070e+05,NaN,RUB,None,None,NaN,None,NaN
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,0,0,None,None,2020-06-23 06:16:18,7703695246,...,2020-12-30,Moskva,3.318084e+07,NaN,RUB,None,None,NaN,None,NaN
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,0,0,None,None,2021-06-09 15:04:20,7813417393,...,2021-12-30,Sankt-Peterburg,2.500000e+07,NaN,RUB,None,None,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,0,0,None,None,2020-06-18 13:00:13,7703695246,...,None,Moskva,5.530140e+06,NaN,RUB,None,None,NaN,None,NaN
1537,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,0,0,None,None,2020-11-18 07:20:58,7728816598,...,2020-12-30,Moskva,1.848000e+05,NaN,RUB,None,None,NaN,None,NaN
1538,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,0,0,None,None,2020-07-03 13:49:05,7703695246,...,2020-12-30,Moskva,3.341800e+08,NaN,RUB,None,None,NaN,None,NaN
1539,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,Krasnodarskii_krai,2.949280e+05,NaN,RUB,None,None,NaN,None,NaN


Необходимо дополнить выборку информацией о позициях договора

In [9]:
covid_contracts_positions_with_lot = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, c.registration_number as contract_registration_number,
    c.notice_not_placed_by_fz223p5s4 as notice_not_placed_by_fz223p5s4,
    c.extension as contract_extension, c.prolongation as contract_prolongation,
    c.customer_appeale_or_needs_approval as customer_appeale_or_needs_approval,
    c.customer_approval_or_antimonopoly_descision_date as customer_approval_or_antimonopoly_descision_date,
    c.plan_position_guid as contract_plan_position_guid,
    c.create_date_time as contract_create_date_time,
    c.customer_inn as contract_customer_inn, c.placer_inn as contract_placer_inn, c.detached_org_inn as contract_detached_org_inn,
    c.publication_date as contract_publication_date, c.contract_status_code as contract_publication_date,
    c.emergency as emergency, c.provider as provider, c.provider_code as provider_code, c.contract_reg_number as contract_reg_number,
    c.contract_date as contract_date, c.digital_purchase as digital_purchase,
    c.purchase_notice_info_guid as purchase_notice_info_guid, c.purchase_notice_info_number as purchase_notice_info_number,
    c.lot_guid as lot_guid, c.subject_contract as subject_contract, c.purchase_type_code as purchase_type_code,
    c.price as contract_price, c.rub_price as contract_rub_price, c.currency_code as contract_currency_code,
    c.start_execution_date as contract_start_execution_date, c.end_execution_date as contract_end_execution_date,
    c.region_name as region_name,
    pnld.initial_sum as notice_lot_initial_sum, pnld.starting_contract_price_rub as notice_lot_starting_contract_price_rub,
    pnld.currency_code as notice_lot_currency_code,
    pnld.commodity_price as notice_lot_commodity_price, pnld.commodity_price_rub as notice_lot_commodity_price_rub,
    pnld.max_contract_price as notice_lot_max_contract_price, pnld.max_contract_price_rub as notice_lot_max_contract_price_rub,
    cp.is_complete as contract_is_complete,
    cpos.name as contract_position_name,
    cpos.okpd2_code as position_okpd2_code, cpos.okpd2_name as position_okpd2_name,
    cpos.country_code as position_country_code, cpos.producer_country as position_producer_country,
    cpos.impossible_to_determine_attr as position_impossible_to_determine_attr, cpos.okei_code as position_okei_code,
    cpos.okei_name as position_okei_name, cpos.qty as position_qty, cpos.unit_price as position_unit_price,
    cpos.rub_unit_price as position_rub_unit_price, cpos.currency_code as position_currency_code
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN purchase_notice_lot_data pnld on pnl.guid = pnld.purchase_notice_lot_guid
    LEFT JOIN (
        SELECT c2.contract_reg_number, MIN(is_complete) as is_complete
        FROM contract c2
        LEFT JOIN contract_performance cp2 ON c2.contract_reg_number = cp2.contract_registration_number
        WHERE c2.emergency = 1
        AND c2.lot_guid is not null
        GROUP BY c2.contract_reg_number
    ) cp ON cp.contract_reg_number = c.contract_reg_number
    LEFT JOIN position_to_contract ptc on c.guid = ptc.contract_guid
    LEFT JOIN contract_position cpos on ptc.position_guid = cpos.guid
    """,
    conn
)

covid_contracts_positions_with_lot

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okpd2_name,position_country_code,position_producer_country,position_impossible_to_determine_attr,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,1,0,0,None,None,2020-09-08 07:01:28,7825011824,...,Работы по монтажу водопроводных и канализацион...,None,None,0,876,Условная единица,1.0,590489.94,NaN,RUB
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,0,0,None,None,2021-08-17 14:12:13,6901067107,...,"Работы электромонтажные прочие, не включенные ...",None,None,1,None,None,NaN,2108040.00,NaN,RUB
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,0,0,None,None,2020-06-05 07:00:39,2312210286,...,Работы строительные по прокладке местных трубо...,None,None,1,None,None,NaN,NaN,NaN,None
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,0,0,None,None,2020-06-23 06:16:18,7703695246,...,Аппараты дыхательные реанимационные,None,None,0,796,Штука,18.0,NaN,NaN,None
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,0,0,None,None,2021-06-09 15:04:20,7813417393,...,"Работы строительные специализированные, не вкл...",None,None,1,None,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,0,0,None,None,2020-06-18 13:00:13,7703695246,...,Аппараты дыхательные реанимационные,None,None,0,796,Штука,3.0,NaN,NaN,None
2045,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,0,0,None,None,2020-11-18 07:20:58,7728816598,...,"Оборудование специального назначения прочее, н...",None,None,0,796,Штука,1.0,NaN,NaN,None
2046,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,0,0,None,None,2020-07-03 13:49:05,7703695246,...,Аппараты дыхательные реанимационные,None,None,0,796,Штука,220.0,NaN,NaN,None
2047,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,Работы строительные по прокладке местных трубо...,None,None,1,None,None,NaN,NaN,NaN,None


Добавим информацию о том, сколько договоров заключено по лоту в выборку.

In [10]:
contracts_count_for_lot = pd.read_sql_query(
    """
    SELECT lot_guid, COUNT(*) as contracts_for_lot
        FROM contract c
        INNER JOIN (
            SELECT contract_reg_number, MAX(version) ver
            FROM contract
            WHERE emergency = 1
            AND lot_guid is not null
            GROUP BY contract_reg_number
        ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    GROUP BY lot_guid
    """,
    conn
)

contracts_count_for_lot

,lot_guid,contracts_for_lot
0,0017ef8e-08d7-4175-a7fd-059f347b92a2,1
1,005056bd-0450-1eeb-89fc-3a3c448d4a86,1
2,005056be-1598-1eda-b6e2-3ac026ca5fa2,1
3,005056be-1598-1edb-8ae4-9691a89e1202,1
4,005056be-1598-1eeb-8ed6-0c774e0f5be6,1
...,...,...
1493,ff359f19-e1d5-4372-8817-bc166a7bdcfb,1
1494,ff46f520-4f87-4c0b-b114-a7527bf40b6d,1
1495,ffd1be89-33ae-4df4-95ed-ab4b7269c2c8,1
1496,ffdd300a-57e0-493f-8f46-460d1ff65276,1


In [23]:
dataset = pd.merge(
    covid_contracts_positions_with_lot,
    contracts_count_for_lot,
    how='left',
    on=['lot_guid']
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_country_code,position_producer_country,position_impossible_to_determine_attr,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,1,0,0,None,None,2020-09-08 07:01:28,7825011824,...,None,None,0,876,Условная единица,1.0,590489.94,NaN,RUB,1
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,0,0,None,None,2021-08-17 14:12:13,6901067107,...,None,None,1,None,None,NaN,2108040.00,NaN,RUB,1
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,0,0,None,None,2020-06-05 07:00:39,2312210286,...,None,None,1,None,None,NaN,NaN,NaN,None,1
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,0,0,None,None,2020-06-23 06:16:18,7703695246,...,None,None,0,796,Штука,18.0,NaN,NaN,None,1
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,0,0,None,None,2021-06-09 15:04:20,7813417393,...,None,None,1,None,None,NaN,NaN,NaN,None,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,0,0,None,None,2020-06-18 13:00:13,7703695246,...,None,None,0,796,Штука,3.0,NaN,NaN,None,1
2045,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,0,0,None,None,2020-11-18 07:20:58,7728816598,...,None,None,0,796,Штука,1.0,NaN,NaN,None,1
2046,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,0,0,None,None,2020-07-03 13:49:05,7703695246,...,None,None,0,796,Штука,220.0,NaN,NaN,None,1
2047,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,None,1,None,None,NaN,NaN,NaN,None,1


Добавим информацию об изначальных заказчиках в выборку

In [19]:
notice_customers_for_contract = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, pn.customer_inn as notice_customer_inn,
        pn.placer_inn as notice_placer_inn, pn.detached_org_inn as notice_detached_org_inn
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN (
        --если несколько извещений с одинаковым номером, выбераем последнее
        SELECT c2.guid, c2.purchase_notice_info_number, MAX(pn2.create_date_time) as max_date
        FROM contract c2
        INNER JOIN purchase_notice pn2 ON pn2.registration_number = c2.purchase_notice_info_number
        WHERE c2.emergency = 1
        AND c2.purchase_notice_info_number is not null
        GROUP BY c2.guid, c2.purchase_notice_info_number
    ) tmp2 ON c.guid = tmp2.guid
    LEFT JOIN purchase_notice pn ON pn.create_date_time = tmp2.max_date
        AND pn.registration_number = tmp2.purchase_notice_info_number
    WHERE c.emergency = 1
    GROUP BY c.guid, pn.customer_inn, pn.placer_inn, pn.detached_org_inn
    """,
    conn
)

notice_customers_for_contract

,contract_guid,notice_customer_inn,notice_placer_inn,notice_detached_org_inn
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,7825011824,7825011824,None
1,00751401-bc7d-40eb-8eb6-191df67b117e,6901067107,6901067107,None
2,00954370-200e-403a-997b-f469f075967e,2312210286,2312210286,None
3,0127411b-0516-4795-989e-38cb1ad9a343,7703695246,7703695246,None
4,013cab73-6528-4279-a181-faed52623a60,7813417393,7813417393,None
...,...,...,...,...
1536,ff80b68f-5be0-4951-b217-fa1aac06905a,7703695246,7703695246,None
1537,ffa42833-0759-40db-ae64-5a329c09199a,7728816598,7728816598,None
1538,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,7703695246,7703695246,None
1539,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,2312210286,2312210286,None


In [24]:
dataset = pd.merge(
    dataset,
    notice_customers_for_contract,
    how='left',
    on='contract_guid'
)

dataset

,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okei_code,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,1,0,0,None,None,2020-09-08 07:01:28,7825011824,...,876,Условная единица,1.0,590489.94,NaN,RUB,1,7825011824,7825011824,None
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,0,0,None,None,2021-08-17 14:12:13,6901067107,...,None,None,NaN,2108040.00,NaN,RUB,1,6901067107,6901067107,None
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,0,0,None,None,2020-06-05 07:00:39,2312210286,...,None,None,NaN,NaN,NaN,None,1,2312210286,2312210286,None
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,0,0,None,None,2020-06-23 06:16:18,7703695246,...,796,Штука,18.0,NaN,NaN,None,1,7703695246,7703695246,None
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,0,0,None,None,2021-06-09 15:04:20,7813417393,...,None,None,NaN,NaN,NaN,None,23,7813417393,7813417393,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,0,0,None,None,2020-06-18 13:00:13,7703695246,...,796,Штука,3.0,NaN,NaN,None,1,7703695246,7703695246,None
2045,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,0,0,None,None,2020-11-18 07:20:58,7728816598,...,796,Штука,1.0,NaN,NaN,None,1,7728816598,7728816598,None
2046,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,0,0,None,None,2020-07-03 13:49:05,7703695246,...,796,Штука,220.0,NaN,NaN,None,1,7703695246,7703695246,None
2047,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,None,NaN,NaN,NaN,None,1,2312210286,2312210286,None


Добавим информацию о том, сколько позиций в каждом контракте

In [21]:
contract_positions_count = pd.read_sql_query(
    """
    SELECT c.guid as contract_guid, COUNT(*) as contract_positions_count
    FROM contract c
    INNER JOIN (
        SELECT contract_reg_number, MAX(version) ver
        FROM contract
        WHERE emergency = 1
        AND lot_guid is not null
        GROUP BY contract_reg_number
    ) tmp ON c.contract_reg_number = tmp.contract_reg_number AND c.version = tmp.ver
    INNER JOIN purchase_notice_lot pnl ON c.lot_guid = pnl.guid
    LEFT JOIN position_to_contract ptc on c.guid = ptc.contract_guid
    LEFT JOIN contract_position cpos on ptc.position_guid = cpos.guid
    GROUP BY c.guid
    """,
    conn
)

contract_positions_count

,contract_guid,contract_positions_count
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,1
1,00751401-bc7d-40eb-8eb6-191df67b117e,1
2,00954370-200e-403a-997b-f469f075967e,1
3,0127411b-0516-4795-989e-38cb1ad9a343,1
4,013cab73-6528-4279-a181-faed52623a60,1
...,...,...
1536,ff80b68f-5be0-4951-b217-fa1aac06905a,1
1537,ffa42833-0759-40db-ae64-5a329c09199a,1
1538,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,1
1539,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,1


In [25]:
dataset = pd.merge(
    dataset,
    contract_positions_count,
    how='left',
    on='contract_guid'
)

dataset


,contract_guid,contract_registration_number,notice_not_placed_by_fz223p5s4,contract_extension,contract_prolongation,customer_appeale_or_needs_approval,customer_approval_or_antimonopoly_descision_date,contract_plan_position_guid,contract_create_date_time,contract_customer_inn,...,position_okei_name,position_qty,position_unit_price,position_rub_unit_price,position_currency_code,contracts_for_lot,notice_customer_inn,notice_placer_inn,notice_detached_org_inn,contract_positions_count
0,0036c56b-1874-4c0c-a7fe-7b8a47696acd,87825011824200000490001,0,1,0,0,None,None,2020-09-08 07:01:28,7825011824,...,Условная единица,1.0,590489.94,NaN,RUB,1,7825011824,7825011824,None,1
1,00751401-bc7d-40eb-8eb6-191df67b117e,56901067107210018890001,0,0,0,0,None,None,2021-08-17 14:12:13,6901067107,...,None,NaN,2108040.00,NaN,RUB,1,6901067107,6901067107,None,1
2,00954370-200e-403a-997b-f469f075967e,52312210286200000710001,0,0,0,0,None,None,2020-06-05 07:00:39,2312210286,...,None,NaN,NaN,NaN,None,1,2312210286,2312210286,None,1
3,0127411b-0516-4795-989e-38cb1ad9a343,67703695246200000980001,0,0,0,0,None,None,2020-06-23 06:16:18,7703695246,...,Штука,18.0,NaN,NaN,None,1,7703695246,7703695246,None,1
4,013cab73-6528-4279-a181-faed52623a60,57813417393210000360003,0,0,0,0,None,None,2021-06-09 15:04:20,7813417393,...,None,NaN,NaN,NaN,None,23,7813417393,7813417393,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,ff80b68f-5be0-4951-b217-fa1aac06905a,67703695246200000500001,0,0,0,0,None,None,2020-06-18 13:00:13,7703695246,...,Штука,3.0,NaN,NaN,None,1,7703695246,7703695246,None,1
2045,ffa42833-0759-40db-ae64-5a329c09199a,57728816598200006640001,0,0,0,0,None,None,2020-11-18 07:20:58,7728816598,...,Штука,1.0,NaN,NaN,None,1,7728816598,7728816598,None,1
2046,ffa4cc42-393b-4774-b2ed-1c9b2fbd75bb,67703695246200001160001,0,0,0,0,None,None,2020-07-03 13:49:05,7703695246,...,Штука,220.0,NaN,NaN,None,1,7703695246,7703695246,None,1
2047,ffd13b8b-ffef-4b0d-8fa5-3214f3e44e2b,52312210286200000990001,0,0,0,0,None,None,2020-12-08 11:26:50,2312210286,...,None,NaN,NaN,NaN,None,1,2312210286,2312210286,None,1


Выгрузим датасет в эксель

In [26]:
dataset.to_excel("output.xlsx")

Проанализируем исполнение договоров в нашей выборке:

In [27]:
failed_contracts = covid_contracts_performance_with_lot[covid_contracts_performance_with_lot["contract_is_complete"] == 0]
completed_contracts = covid_contracts_performance_with_lot[covid_contracts_performance_with_lot["contract_is_complete"] == 1]

contracts_count = covid_contracts_performance_with_lot.shape[0]
failed_contracts_count = failed_contracts.shape[0]
completed_contracts_count = completed_contracts.shape[0]

print(f'Всего договоров: {contracts_count}')
print(f'Всего завершенных договоров: {failed_contracts_count + completed_contracts_count}')
print(f'Всего расторженных договоров: {failed_contracts_count}')
print(f'Всего исполненных договоров: {completed_contracts_count}')
print(f'Процент расторженных договоров от завершенных: {failed_contracts_count / (failed_contracts_count + completed_contracts_count) * 100}%')
print(f'Процент расторженных договоров от всех: {failed_contracts_count / contracts_count * 100}%')

Всего договоров: 1541
Всего завершенных договоров: 830
Всего расторженных договоров: 42
Всего исполненных договоров: 788
Процент расторженных договоров от завершенных: 5.0602409638554215%
Процент расторженных договоров от всех: 2.7255029201817003%


Проанализируем в каком количестве позиций присутствует цена за единицу

In [28]:
positions_count = covid_contracts_positions_with_lot.shape[0]
unit_price_positions_count = covid_contracts_positions_with_lot[covid_contracts_positions_with_lot["position_unit_price"].notnull()].shape[0]

print(f'Всего позиций: {positions_count}')
print(f'Всего позиций с указанной ценой: {unit_price_positions_count}')
print(f'Процент позиций с указанной ценой: {unit_price_positions_count / positions_count * 100}%')

Всего позиций: 2049
Всего позиций с указанной ценой: 537
Процент позиций с указанной ценой: 26.207906295754025%
